In [1]:
# conda activate anndata

import os
import scvi
import torch
import pandas as pd
import scanpy as sc
import anndata as ad


torch.set_num_threads(20)
# OMP_NUM_THREADS=20

pd.set_option("display.max_columns", None)

os.chdir("/mnt/lareaulab/reliscu/projects/NSF_GRFP/analyses/pseudobulk_test/tasic_2018/mouse_ALM")

/mnt/lareaulab/reliscu/anaconda3/envs/anndata/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


NameError: name 'os' is not defined

Here I run scVI to get a latent representation of the single-cell data for visualization purposes

In [ ]:
adata = ad.read_h5ad("/mnt/lareaulab/reliscu/projects/NSF_GRFP/data/scRNA-seq/tasic_2018/ALM/tasic_2018_ALM_STAR_gene_counts.h5ad")

In [ ]:
adata.layers["counts"] = adata.X.copy()
adata.raw = adata  # Keep full dimension safe

In [ ]:
# Subset data to highly variable features

sc.pp.highly_variable_genes(
  adata,
  n_top_genes = 2000,
  subset = True,
  layer = "counts",
  flavor = "seurat_v3",
)

In [ ]:
# scVI model

# Note: this takes about 30 minutes
scvi.model.SCVI.setup_anndata(adata, layer="counts")
model = scvi.model.SCVI(adata)
model.train()

In [ ]:
# Extract low dim rep.
latent = model.get_latent_representation()
adata.obsm["scVI"] = latent

# Save
model.save("data/tasic_2018_ALM_STAR_model", overwrite=True)
adata.write("data/tasic_2018_ALM_STAR_model/tasic_2018_ALM_STAR_gene_counts_scVI.h5ad")

In [ ]:
adata = ad.read_h5ad("data/tasic_2018_ALM_STAR_model/tasic_2018_ALM_STAR_gene_counts_scVI.h5ad")

In [ ]:
model = scvi.model.SCVI.load("data/tasic_2018_ALM_STAR_model", adata=adata)

In [ ]:
de_results_neighborhood = model.differential_expression(
    groupby="neighborhood_label",  # Column in `adata.obs` defining clusters
    mode="change"
)